In [141]:
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns = 999

# Airbnb Data Cleaning and Transformation

In [142]:
url = r"C:\Users\rergu\OneDrive\IronHack\Projects\London Airbnb\listings.csv"

df = pd.read_csv(url, index_col=False)

In [ ]:
# drop columns

df.drop(columns=['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time','host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_group_cleansed', 'bathrooms_text', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy',
       'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'first_review', 'last_review', 'license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'host_response_rate', 'host_acceptance_rate', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'instant_bookable','review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value','property_type', 'host_is_superhost' ], axis=1, inplace=True)

In [144]:
# drop NA rows 
df.dropna(subset=['price'], inplace=True)
df.dropna(subset=['bedrooms'], inplace=True)

In [145]:
# fill NA rows

df.fillna({'review_scores_rating': 0, 'reviews_per_month': 0, 'beds': df['bedrooms'], 'bathrooms': df['bedrooms']}, inplace=True)


In [146]:
#we will only focus on entire home/apt rentals so we drop the other type of roomtypes
df = df[df['room_type'] == 'Entire home/apt']

In [147]:
# remove $ sign from price, the price is local currency which is British Pounds, and convert to float
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

<>:2: SyntaxWarning:

invalid escape sequence '\$'

<>:2: SyntaxWarning:

invalid escape sequence '\$'

C:\Users\rergu\AppData\Local\Temp\ipykernel_10372\3440122836.py:2: SyntaxWarning:

invalid escape sequence '\$'



In [148]:
# id convert to string
df['id'] = df['id'].astype(str)

In [150]:
# fill 0 beds with the average number of beds per bedroom then round to the nearest integer
mask = df[df['beds']!=0]
df['beds'] = df.apply(lambda x: round(mask['beds'].sum()/mask['bedrooms'].sum()) if x['beds'] == 0 else x['beds'], axis=1)

In [151]:
df['bathrooms'] = df.apply(lambda x: 1 if x['bathrooms'] == 0 else x['bathrooms'], axis=1)

In [152]:
df['bathrooms'] = df['bathrooms'].apply(np.ceil)
df['accommodates'] = df['accommodates'].apply(np.ceil)
df['bedrooms'] = df['bedrooms'].apply(np.ceil)
df['beds'] = df['beds'].apply(np.ceil)

In [153]:
# drop rows with bedrooms equal and bigger than 5
df = df[df['bedrooms'] < 5]

# drop rows with beds equal and bigger than 6
df = df[df['beds'] <= 6]

# Remove listings where accommodates bigger than 10
df = df[df['accommodates'] <= 10]

In [ ]:
# convert amenities from json to list
import ast

df['amenities'] = df['amenities'].apply(ast.literal_eval)

In [154]:
# reset index
df.reset_index(drop=True, inplace=True)

In [169]:
df.sample(5)

,id,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,number_of_reviews,review_scores_rating,reviews_per_month,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,cluster_18,cluster_19,weighted_amenities_bin
26593,1176698331729347367,Haringey,51.575678,-0.109262,Entire home/apt,8.0,3.0,4.0,6.0,"[Changing table - available upon request, BBQ ...",315.0,1,5.00,0.15,0,1,1,1,0,0,1,0,1,1,0,0,0,1,0,0,1,1,1,0,high
18894,978002222746080316,Kingston upon Thames,51.333364,-0.317666,Entire home/apt,2.0,1.0,0.0,1.0,"[Private entrance, Dedicated workspace, Stove,...",257.0,104,4.99,5.70,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,1,1,high
27668,1196437910212036301,Hammersmith and Fulham,51.506974,-0.224862,Entire home/apt,4.0,1.0,1.0,2.0,"[Clothing storage: closet, Stove, Iron, Cleani...",78.0,28,4.11,3.73,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,medium
26876,1181055681436728072,Bexley,51.509617,0.129330,Entire home/apt,2.0,1.0,1.0,1.0,"[Mini fridge, Self check-in, Carbon monoxide a...",64.0,22,4.91,2.64,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,high
13202,780554141801354791,Hammersmith and Fulham,51.493680,-0.231600,Entire home/apt,4.0,2.0,2.0,2.0,"[Private entrance, Dryer, Dedicated workspace,...",186.0,19,4.95,1.05,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,1,1,1,1,1,high


In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39385 entries, 0 to 39384
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      39385 non-null  object 
 1   neighbourhood_cleansed  39385 non-null  object 
 2   latitude                39385 non-null  float64
 3   longitude               39385 non-null  float64
 4   room_type               39385 non-null  object 
 5   accommodates            39385 non-null  float64
 6   bathrooms               39385 non-null  float64
 7   bedrooms                39385 non-null  float64
 8   beds                    39385 non-null  float64
 9   amenities               39385 non-null  object 
 10  price                   39385 non-null  float64
 11  number_of_reviews       39385 non-null  int64  
 12  review_scores_rating    39385 non-null  float64
 13  reviews_per_month       39385 non-null  float64
dtypes: float64(9), int64(1), object(4)
mem

In [164]:
df.to_csv(r"C:\Users\rergu\OneDrive\IronHack\Projects\London Airbnb\airbnb-flask-app\df_cleaned.csv", index=False)

## Amenities

In [158]:
# df['amenities'] is already a list of strings for each row

unique_amenities = set(amenity for sublist in df['amenities'] for amenity in sublist)

# If you want them as a sorted list:
amenities_list = sorted(unique_amenities)

In [159]:
amenities_list

[' Bosch stainless steel double oven',
 ' Bosch stainless steel stove',
 ' Bose  sound system with Bluetooth and aux',
 ' Faith In Nature  body soap',
 ' Faith In Nature  conditioner',
 ' Faith In Nature  shampoo',
 ' Gaggenau Metro 90cm avant garde oven stainless steel single oven',
 ' Gaggenau Wine fridge and Bosch undercounter fridge refrigerator',
 ' Gilchrist Soames  body soap',
 ' Gilchrist Soames  conditioner',
 ' Gilchrist Soames  shampoo',
 ' Gilchrist and Soames body soap',
 ' Gilchrist and Soames conditioner',
 ' Gilchrist and Soames shampoo',
 ' Herbal conditioner',
 ' Kinsey Apothecary  body soap',
 ' Kinsey Apothecary  conditioner',
 ' Kinsey Apothecary  shampoo',
 ' Nature body soap',
 ' Smeg refrigerator',
 ' Various  conditioner',
 ' Zanussi stainless steel electric stove',
 ' bertazzoni stainless steel oven',
 ' ‘Ruark’ deluxe Bluetooth radio Bluetooth sound system',
 '"Various" body soap',
 '"Various" conditioner',
 '"Various" shampoo',
 '"portable stove" electric st

In [160]:
# Automatically group amenities by similarity using their names.

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd

# 1. List of unique amenities
#amenities_list = list(df['amenities'])


# 2. TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(amenities_list)

# 3. Clustering (pick number of clusters; e.g., 20 or whatever works for your use case)
num_clusters = 20
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(X)

# 4. Map each amenity to a cluster
amenity_to_cluster = pd.DataFrame({
    'amenity': amenities_list,
    'cluster': clusters
})

# Create a dictionary for fast lookup: amenity -> cluster
cluster_dict = dict(zip(amenity_to_cluster['amenity'], amenity_to_cluster['cluster']))


In [161]:
'''Summary of each cluster with suggested words to use'''
from collections import Counter

def get_cluster_keywords(amenities_in_cluster, topn=3):
    words = []
    for a in amenities_in_cluster:
        words.extend(a.lower().split())
    most_common = Counter(words).most_common(topn)
    return ', '.join([w[0] for w in most_common])

for c in sorted(amenity_to_cluster['cluster'].unique()):
    amenities_in_cluster = amenity_to_cluster[amenity_to_cluster['cluster'] == c]['amenity']
    suggested = get_cluster_keywords(amenities_in_cluster)
    print(f"Cluster {c}: {suggested}")

Cluster 0: with, hdtv, inch
Cluster 1: available, housekeeping, -
Cluster 2: wifi, –, mbps
Cluster 3: hdtv, inch, coffee
Cluster 4: tbc, body, soap
Cluster 5: penhaligons, body, soap
Cluster 6: oven, stainless, steel
Cluster 7: conditioner, in, faith
Cluster 8: -, high, chair
Cluster 9: stove, stainless, steel
Cluster 10: bluetooth, system, sound
Cluster 11: with, hdtv, cable,
Cluster 12: with, amazon, prime
Cluster 13: body, soap, and
Cluster 14: tv,, with, amazon
Cluster 15: sound, system, bluetooth
Cluster 16: tv, with, inch
Cluster 17: refrigerator, fridge, freezer
Cluster 18: premises, on, parking
Cluster 19: shampoo, in, and


In [167]:
df.columns

Index(['id', 'neighbourhood_cleansed', 'latitude', 'longitude', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price',
       'number_of_reviews', 'review_scores_rating', 'reviews_per_month',
       'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14',
       'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19'],
      dtype='object')

In [ ]:
# based on the clusters, run random forest model to create weighted amenities bin as for low, medium, high
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# Create a binary feature for each cluster
#for c in sorted(amenity_to_cluster['cluster'].unique()):
#    df[f'cluster_{c}'] = df['amenities'].apply(lambda x: 1 if any(amenity_to_cluster[amenity_to_cluster['cluster'] == c]['amenity'].isin(x)) else 0)
# Define features and target
X = df.drop(columns=['price', 'amenities', 'neighbourhood_cleansed', 'latitude', 'longitude', 'room_type', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month'])
y = df['price']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   
# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
# Get feature importances
importances = rf.feature_importances_
# Create a DataFrame for feature importances
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)


# create weighted amenities bin based on the feature importances low, medium, high
def get_weighted_amenities_bin(row):
    weighted_bin = 0
    for feature in feature_importances['feature']:
        if row[feature] == 1:
            weighted_bin += feature_importances[feature_importances['feature'] == feature]['importance'].values[0]
    if weighted_bin < 0.1:
        return 'low'
    elif weighted_bin < 0.3:
        return 'medium'
    else:
        return 'high'
# Apply the function to create a new column
df['weighted_amenities_bin'] = df.apply(get_weighted_amenities_bin, axis=1)



# Real Estate Avg Selling Price

### Avg Price csv

In [170]:
df_avg_price = pd.read_csv(r"C:\Users\rergu\OneDrive\IronHack\Projects\London Airbnb\avg_price.csv", index_col=False)

In [171]:
df_avg_price.head(5)

,Date,E09000001,E09000002,E09000003,E09000004,E09000005,E09000006,E09000007,E09000008,E09000009,E09000010,E09000011,E09000012,E09000013,E09000014,E09000015,E09000016,E09000017,E09000018,E09000019,E09000020,E09000021,E09000022,E09000023,E09000024,E09000025,E09000026,E09000027,E09000028,E09000029,E09000030,E09000031,E09000032,E09000033,E13000001,E13000002,E12000001,E12000002,E12000003,E12000004,E12000005,E12000006,E12000007,E12000008,E12000009,E92000001
0,Jan-95,"90,347","51,870","98,948","64,956","76,880","83,082","119,775","70,118","85,469","77,018","65,784","63,375","129,363","81,997","86,737","68,980","75,819","82,041","92,577","197,675","85,184","72,550","64,192","87,732","55,939","70,909","115,973","78,355","72,321","65,716","63,315","101,607","141,417","85,892","76,196","39,809","40,907","42,171","43,856","46,470","56,098","79,687","64,502","52,799","50,231"
1,Feb-95,"81,213","52,513","98,848","64,786","77,651","83,068","118,365","69,908","86,551","77,698","64,405","65,330","126,448","84,806","85,331","70,394","77,047","80,701","94,404","197,297","85,557","69,992","64,593","85,500","55,536","70,862","117,858","74,793","71,670","68,409","62,213","101,701","139,762","83,294","76,174","40,278","40,877","41,912","44,344","47,249","55,991","77,913","64,196","52,462","50,130"
2,Mar-95,"78,168","52,701","97,848","64,366","77,644","82,856","119,131","69,666","87,067","76,674","64,811","65,750","124,944","84,398","85,352","70,368","76,182","81,889","93,527","197,874","85,432","71,014","63,976","86,226","55,854","71,216","113,851","75,906","71,077","70,188","62,852","99,961","140,604","84,069","75,941","40,086","41,351","42,544","43,701","47,345","55,574","79,110","64,597","51,716","50,229"
3,Apr-95,"76,172","54,618","96,273","64,276","78,668","82,525","118,948","69,562","87,933","75,880","65,391","67,349","125,761","85,499","85,507","70,445","75,897","81,653","93,406","199,279","86,024","71,070","63,109","85,508","56,922","70,948","113,373","72,802","70,173","72,706","62,956","99,848","141,784","84,354","75,813","39,827","41,195","42,934","44,414","47,359","55,966","79,708","65,111","52,877","50,597"
4,May-95,"83,392","54,524","95,737","63,995","79,464","82,951","118,781","69,800","87,813","75,017","67,065","68,439","128,877","85,314","85,800","69,522","75,082","82,775","94,408","207,174","86,498","73,903","62,133","86,453","58,305","71,083","114,256","74,356","70,524","72,926","63,496","99,265","142,359","86,854","75,981","40,935","41,154","42,197","44,177","48,234","55,879","80,754","65,017","52,793","50,679"


In [172]:
# convert columns to float from column index 1 to 44 in df_avg_price
for col in df_avg_price.columns[1:]:
    df_avg_price[col] = df_avg_price[col].replace({'\$': '', ',': ''}, regex=True).astype(float)

<>:3: SyntaxWarning:

invalid escape sequence '\$'

<>:3: SyntaxWarning:

invalid escape sequence '\$'

C:\Users\rergu\AppData\Local\Temp\ipykernel_10372\1789840795.py:3: SyntaxWarning:

invalid escape sequence '\$'



In [173]:
# Replace 'Sept' with 'Sep' to match pandas' expected abbreviation
df_avg_price["Date"] = df_avg_price["Date"].str.replace("Sept-", "Sep-", regex=False)
df_avg_price["Date"] = pd.to_datetime(df_avg_price["Date"], format='%b-%y')


In [174]:
df_avg_price['year'] = df_avg_price['Date'].dt.year
df_avg_price['month'] = df_avg_price['Date'].dt.month

In [175]:
df_avg_price.columns

Index(['Date', 'E09000001', 'E09000002', 'E09000003', 'E09000004', 'E09000005',
       'E09000006', 'E09000007', 'E09000008', 'E09000009', 'E09000010',
       'E09000011', 'E09000012', 'E09000013', 'E09000014', 'E09000015',
       'E09000016', 'E09000017', 'E09000018', 'E09000019', 'E09000020',
       'E09000021', 'E09000022', 'E09000023', 'E09000024', 'E09000025',
       'E09000026', 'E09000027', 'E09000028', 'E09000029', 'E09000030',
       'E09000031', 'E09000032', 'E09000033', 'E13000001', 'E13000002',
       'E12000001', 'E12000002', 'E12000003', 'E12000004', 'E12000005',
       'E12000006', 'E12000007', 'E12000008', 'E12000009', 'E92000001', 'year',
       'month'],
      dtype='object')

In [ ]:
df_avg_price.drop(columns=['NORTH EAST',
       'NORTH WEST', 'YORKS and  THE HUMBER', 'EAST MIDLANDS', 'WEST MIDLANDS',
       'EAST OF ENGLAND', 'SOUTH EAST', 'SOUTH WEST', 'England',], inplace=True)

In [176]:
# import column names

df_legend = pd.read_csv(r"C:\Users\rergu\OneDrive\IronHack\Projects\London Airbnb\price_legend.csv", index_col=False)

In [177]:
# Create a mapping: id -> neighborhood
id_to_name = dict(zip(df_legend['id'], df_legend['neighborhood']))

# For each column in df_avg_price, replace if in mapping
df_avg_price = df_avg_price.rename(columns=id_to_name)


In [178]:
grouped_avg_price = df_avg_price.groupby(['year']).mean().reset_index()
grouped_avg_price.drop(columns=['month', 'Date'], inplace=True)

In [179]:
grouped_avg_price = grouped_avg_price[grouped_avg_price['year'] == 2024]

In [180]:

grouped_avg_price.drop(columns=['year'], inplace=True)

In [181]:
# transpose the dataframes

df_price = grouped_avg_price.transpose()

In [182]:

df_price.reset_index(inplace=True)

In [183]:

df_price.rename(columns={'index': 'neighborhood'}, inplace=True)

In [184]:
#rename  29 to price_index and avg_price_2024
df_price.rename(columns={29: 'avg_selling_price_2024'}, inplace=True)

### Merge airbnb and real estate price

In [187]:
# Step 1: Standardize both columns in each DataFrame

def clean_name(s):
    return (
        s.lower()
        .replace('&', 'and')
        .replace('-', ' ')
        .replace('/', ' ')
        .replace('.', '')
        .replace(',', '')
        .replace('  ', ' ')
        .strip()
    )

# Clean in both DataFrames
df['neighbourhood_cleaned'] = df['neighbourhood_cleansed'].apply(clean_name)
df_price['neighborhood_cleaned'] = df_price['neighborhood'].apply(clean_name)

# Step 2: Merge on the cleaned columns
df_final = pd.merge(
    df,
    df_price,
    left_on='neighbourhood_cleaned',
    right_on='neighborhood_cleaned',
    how='left'
)


In [188]:
df_final.drop(columns=['neighbourhood_cleansed', 'neighborhood_cleaned','neighborhood'], inplace=True)

In [189]:
df_final['room_type'].replace({'Entire home/apt': 'entire_home'}, inplace=True)

C:\Users\rergu\AppData\Local\Temp\ipykernel_10372\4076650813.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [190]:
df_final.reset_index(drop=True, inplace=True)

In [191]:
df_neighborhood = pd.read_csv(r'C:\Users\rergu\OneDrive\IronHack\Projects\London Airbnb\neighborhood_group.csv', index_col=False)

In [192]:
df_neighborhood.head()

,Borough,Group
0,City of London,Central
1,Barking and Dagenham,East
2,Barnet,North
3,Bexley,East
4,Brent,West


In [193]:
df_final['price_tier'] = pd.qcut(df_final['avg_selling_price_2024'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

'''
Low < 565889.0
Medium < 617720.0
High < 842340.0
Very High >= 842340.0

'''


'\nLow < 565889.0\nMedium < 617720.0\nHigh < 842340.0\nVery High >= 842340.0\n\n'

In [194]:
def clean_name(s):
    return (
        s.lower()
        .replace('&', 'and')
        .replace('-', ' ')
        .replace('/', ' ')
        .replace('.', '')
        .replace(',', '')
        .replace('  ', ' ')
        .strip()
    )

# Clean in both DataFrames
df_neighborhood['Borough'] = df_neighborhood['Borough'].apply(clean_name)

# Step 2: Merge on the cleaned columns
df_x = pd.merge(
    df_final,
    df_neighborhood,
    left_on='neighbourhood_cleaned',
    right_on='Borough',
    how='left'
)


In [212]:
df_x.head()

,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month,weighted_amenities_bin,neighbourhood_cleaned,avg_selling_price_2024,price_tier,borough_group
0,51.480890,-0.147750,entire_home,2.0,1.0,1.0,1.0,94.0,25,4.23,0.15,high,wandsworth,7.193749e+05,High,South
1,51.551040,-0.083240,entire_home,2.0,1.0,1.0,1.0,86.0,317,4.84,1.90,high,islington,7.065026e+05,Medium,Central
2,51.499930,-0.217070,entire_home,5.0,1.0,2.0,3.0,175.0,40,4.85,0.25,high,hammersmith and fulham,8.249464e+05,High,West
3,51.538550,-0.135920,entire_home,4.0,2.0,1.0,2.0,153.0,195,4.82,1.25,high,camden,8.423398e+05,High,Central
4,51.518624,-0.195871,entire_home,4.0,2.0,2.0,3.0,350.0,26,4.68,0.15,high,westminster,1.071103e+06,High,Central


In [196]:
df_x.rename(columns={'Group': 'borough_group'}, inplace=True)

In [198]:
df_x.drop(columns='Borough', inplace=True)

In [207]:
df_x.drop(columns=['id', 'amenities', 'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14',
       'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19'], axis=1, inplace=True)

In [209]:
#drop these outliers
df_x = df_x[df_x['price'] < df_x['price'].quantile(0.9)]

In [211]:
df_x.to_csv(r"C:\Users\rergu\OneDrive\IronHack\Projects\London Airbnb\airbnb-flask-app\df_final.csv", index=False)